In [63]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader

In [64]:
df = pd.read_csv('defillama_stablecoin_pools.csv')

# for `APY Reward`, `APY Base`, `APY`, and `APY Mean 30d` column, fill na with 0
df['APY Reward'] = df['APY Reward'].fillna(0)
df['APY Base'] = df['APY Base'].fillna(0)
df['APY'] = df['APY'].fillna(0)
df['APY Mean 30d'] = df['APY Mean 30d'].fillna(0)

# for `Confidence` column, fill na with 1
df['Confidence'] = df['Confidence'].fillna(1)

# for `Outlook` column, fill na with "Down"
df['Outlook'] = df['Outlook'].fillna('Down')

In [65]:
# Check na
print(df.isna().sum())

Pool            0
Project         0
Category        0
Chain           0
TVL             0
APY             0
APY Base        0
APY Mean 30d    0
APY Reward      0
Outlook         0
Confidence      0
dtype: int64


In [66]:
categorical_cols = df.select_dtypes(include=['object']).columns

# One-hot encode categorical columns
df = pd.get_dummies(df, columns=categorical_cols)
df.columns

Index(['TVL', 'APY', 'APY Base', 'APY Mean 30d', 'APY Reward', 'Confidence',
       'Pool_9SUSDC11CORE', 'Pool_9SUSDCCORE', 'Pool_ADAI-AUSDC-AUSDT',
       'Pool_ALUSD-FRAX-USDC',
       ...
       'Chain_Sui', 'Chain_Taiko', 'Chain_Tezos', 'Chain_Ton', 'Chain_Tron',
       'Chain_Unit0', 'Chain_Venom', 'Chain_zkSync Era', 'Outlook_Down',
       'Outlook_Up'],
      dtype='object', length=623)

In [67]:
X = df.drop('Confidence', axis=1)
y = df['Confidence'] - 1

# Normalize features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(X.shape, y.shape)

(1160, 622) (1160,)


In [68]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train.values)  # assuming y is integer class labels
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test.values)

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [69]:
class ANN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Set dimensions based on your data
input_dim = X_train_tensor.shape[1]
hidden_dim = 64  # You can adjust this as needed
output_dim = len(np.unique(y))  # Assumes 'target' contains class labels

model = ANN(input_dim, hidden_dim, output_dim)
print(model)

ANN(
  (fc1): Linear(in_features=622, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=3, bias=True)
)


In [70]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [71]:
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()  # Set model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()         # Zero the parameter gradients
            outputs = model(inputs)       # Forward pass
            loss = criterion(outputs, labels)
            loss.backward()               # Backpropagation
            optimizer.step()              # Update parameters
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

def evaluate(model, test_loader):
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Compute accuracy
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    accuracy = np.mean(all_preds == all_labels)
    
    print(f"Accuracy: {accuracy:.4f}")
    return all_labels, all_preds


In [72]:
num_epochs = 100  # Adjust the number of epochs as needed
train(model, train_loader, criterion, optimizer, num_epochs)

Epoch 1/100, Loss: 1.0565
Epoch 2/100, Loss: 0.8696
Epoch 3/100, Loss: 0.7067
Epoch 4/100, Loss: 0.5776
Epoch 5/100, Loss: 0.4899
Epoch 6/100, Loss: 0.4390
Epoch 7/100, Loss: 0.3979
Epoch 8/100, Loss: 0.3707
Epoch 9/100, Loss: 0.3548
Epoch 10/100, Loss: 0.3358
Epoch 11/100, Loss: 0.3241
Epoch 12/100, Loss: 0.3124
Epoch 13/100, Loss: 0.3023
Epoch 14/100, Loss: 0.2871
Epoch 15/100, Loss: 0.2810
Epoch 16/100, Loss: 0.2751
Epoch 17/100, Loss: 0.2650
Epoch 18/100, Loss: 0.2571
Epoch 19/100, Loss: 0.2514
Epoch 20/100, Loss: 0.2429
Epoch 21/100, Loss: 0.2376
Epoch 22/100, Loss: 0.2279
Epoch 23/100, Loss: 0.2225
Epoch 24/100, Loss: 0.2191
Epoch 25/100, Loss: 0.2078
Epoch 26/100, Loss: 0.2042
Epoch 27/100, Loss: 0.1984
Epoch 28/100, Loss: 0.1933
Epoch 29/100, Loss: 0.1850
Epoch 30/100, Loss: 0.1811
Epoch 31/100, Loss: 0.1758
Epoch 32/100, Loss: 0.1737
Epoch 33/100, Loss: 0.1678
Epoch 34/100, Loss: 0.1642
Epoch 35/100, Loss: 0.1623
Epoch 36/100, Loss: 0.1553
Epoch 37/100, Loss: 0.1533
Epoch 38/1

In [73]:
true_labels, predictions = evaluate(model, test_loader)

# Print the classification report and confusion matrix
print("Classification Report:")
print(classification_report(true_labels, predictions))

print("Confusion Matrix:")
print(confusion_matrix(true_labels, predictions))

Accuracy: 0.4569
Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.72      0.57        88
           1       0.39      0.24      0.30        83
           2       0.47      0.38      0.42        61

    accuracy                           0.46       232
   macro avg       0.45      0.44      0.43       232
weighted avg       0.44      0.46      0.43       232

Confusion Matrix:
[[63 16  9]
 [46 20 17]
 [23 15 23]]


In [ ]:
# Export the model
torch.save(model.state_dict(), '../model/risk-assess-model.pth')